In [1]:
print([[] for _ in range(9)])
print([])
print()

[[], [], [], [], [], [], [], [], []]
[]


In [ ]:
apiVersion: "kubeflow.org/v1"
kind: PyTorchJob
metadata:
  name: belle-dist-torchrun
  namespace: mlops-gomall
spec:
  pytorchReplicaSpecs:
    Master: #一个机子两张40GA100 共一个master
      replicas: 1
      restartPolicy: OnFailure
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
            #到此为止不用改
        spec:
          volumes:
            - name: modelpv #好像也不用改
              persistentVolumeClaim:
                claimName: pretrained-models-volume
            - name: codepv
              persistentVolumeClaim:
                claimName: code-belle-volume
          containers:
            - name: pytorch
              image: registry-ccloud.conestore.cn:8443/chatgpt/torch:v0.0.3 #镜像也不用改
              imagePullPolicy: Always
              volumeMounts:
                - mountPath: /model #卷的位置
                  name: modelpv #将模型挂载到卷的位置
                - mountPath: /code #代码同理
                  name: codepv
              resources:
                limits:
                  cpu: 16
                  nvidia.com/gpu: 2
              command:
                - "sh"
                - "/code/stanford_alpaca/train_belle.sh"
    Worker: #一个机子两张40GA100 共4个worker
      replicas: 2
      restartPolicy: OnFailure
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          volumes:
            - name: modelpv
              persistentVolumeClaim:
                claimName: pretrained-models-volume
            - name: codepv
              persistentVolumeClaim:
                claimName: code-belle-volume
          containers:
            - name: pytorch
              image: registry-ccloud.conestore.cn:8443/chatgpt/torch:v0.0.3
              imagePullPolicy: Always
              volumeMounts:
                - mountPath: /model
                  name: modelpv
                - mountPath: /code
                  name: codepv
              resources:
                limits:
                  cpu: 16
                  nvidia.com/gpu: 2
              command: #运行的命令
                - "sh"
                - "/code/stanford_alpaca/train_belle.sh"
